In [1]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64
import random

In [2]:
df = pd.read_csv("data/kidney.csv")
df.pop("target")

columns = df.columns.values
rand = random.randint(6, len(columns))

features = df.values[rand]

inputs = {key: value for key, value in zip(columns, features)}
inputs

{'gravity': 1.008,
 'ph': 5.09,
 'osmo': 371.0,
 'cond': 15.5,
 'urea': 159.0,
 'calc': 2.17}

In [3]:
def float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(
            value=[value]
        ),
    )
    
def int_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(
            value=[value]
        ),
    )

In [4]:
def prepare_json(inputs: dict):
    feature_spec = dict()
    
    for keys, values in inputs.items():
        if keys in ('osmo','urea'):
            values=int(values)
            feature_spec[keys] = int_feature(values)
        elif isinstance(values, float):
            feature_spec[keys] = float_feature(values)
            
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
        
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

In [5]:
def make_predictions(inputs):
    json_data = prepare_json(inputs)
    
    endpoint = "http://localhost:8080/v1/models/cc-model:predict"
    response = requests.post(endpoint, data=json_data)

    prediction = response.json()["predictions"][0][0]
    return "kidney stone " if prediction > 0.6 else "no kidney stone"

In [6]:
make_predictions(inputs)

'kidney stone '